<a href="https://colab.research.google.com/github/marianatiele/rdata_Enem2023/blob/main/ETL_ENEM2023_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Link da Base de Dados
- [Dados Abertos - Sisu Chamada Regular 2023.2](https://dadosabertos.mec.gov.br/images/conteudo/sisu/2023/chamada_regular_sisu_2023_2.csv)

## Criando o Ambiente PySpark

In [ ]:
!apt-get update -qq
!apt-get install openjdk-8-jdk-headless -qq > /dev/null  # Instalação do Java

# Baixar a versão do Spark desejada (Spark 3.5.4 com Hadoop 3)
!wget -q https://archive.apache.org/dist/spark/spark-3.5.4/spark-3.5.4-bin-hadoop3.tgz

# Extrair os arquivos do Spark
!tar xf spark-3.5.4-bin-hadoop3.tgz


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [ ]:
# Variáveis de ambiente

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.4-bin-hadoop3"

In [ ]:
!pip install findspark

In [ ]:
import findspark
findspark.init()

In [ ]:
!pip install pyngrok
from pyngrok import ngrok

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[*]') \
    .appName("Iniciando com Spark") \
    .config('spark.ui.port', '4050') \
    .getOrCreate()

In [ ]:
from pyspark.sql.functions import *

## Obtendo Os Dados

In [ ]:
path = '/content/chamada_regular_sisu_2023_2.csv'

In [ ]:
dados = (spark.read.format('csv').option('sep', '|')
        .option('header', 'true').option('inferSchema', 'true')
        .option('encoding', 'iso-8859-1').load(path))

In [ ]:
dados.count(), len(dados.columns)

(578781, 56)

In [ ]:
dados.select('*').show()

+----+------+-----+---------------+----------+--------------------+---------+------+-------------+--------------------+---------+----------------+------------+--------------------+-----------+-------+----------------+-------+---------------------+------------------+---------------------+----------------+------+-------+-------+------+------+-------------+--------------+--------------+-------------+-------------+------------+-------------+--------------+--------------------+----+-------------+------------+-------------------+-----+------+-------+-------+------+------+---------------+----------------+----------------+---------------+---------------+--------------+----------+-------------+--------+--------------+
| ANO|EDICAO|ETAPA|       DS_ETAPA|CODIGO_IES|            NOME_IES|SIGLA_IES|UF_IES|CODIGO_CAMPUS|         NOME_CAMPUS|UF_CAMPUS|MUNICIPIO_CAMPUS|CODIGO_CURSO|          NOME_CURSO|       GRAU|  TURNO|DS_PERIODICIDADE|TP_COTA|TIPO_MOD_CONCORRENCIA|  MOD_CONCORRENCIA|QT_VAGAS_CONCOR

In [ ]:
from pyspark.sql.functions import *

In [ ]:
dados.select([count(when(col(c).isNull(), c)).alias(c) for c in dados.columns]).show()

+---+------+-----+--------+----------+--------+---------+------+-------------+-----------+---------+----------------+------------+----------+----+-----+----------------+-------+---------------------+----------------+---------------------+----------------+------+-------+-------+------+------+-------------+--------------+--------------+-------------+-------------+------------+---+--------------+--------+----+-------------+------------+-------------------+-----+------+-------+-------+------+------+---------------+----------------+----------------+---------------+---------------+--------------+----------+-------------+--------+---------+
|ANO|EDICAO|ETAPA|DS_ETAPA|CODIGO_IES|NOME_IES|SIGLA_IES|UF_IES|CODIGO_CAMPUS|NOME_CAMPUS|UF_CAMPUS|MUNICIPIO_CAMPUS|CODIGO_CURSO|NOME_CURSO|GRAU|TURNO|DS_PERIODICIDADE|TP_COTA|TIPO_MOD_CONCORRENCIA|MOD_CONCORRENCIA|QT_VAGAS_CONCORRENCIA|PERCENTUAL_BONUS|PESO_L|PESO_CH|PESO_CN|PESO_M|PESO_R|NOTA_MINIMA_L|NOTA_MINIMA_CH|NOTA_MINIMA_CN|NOTA_MINIMA_M|NOTA_MIN

In [ ]:
coluns = ['ANO', 'NOME_IES','UF_IES', 'NOME_CAMPUS', 'UF_CAMPUS',
           'MUNICIPIO_CAMPUS', 'NOME_CURSO', 'GRAU', 'TURNO',
       'DS_PERIODICIDADE', 'TIPO_MOD_CONCORRENCIA', 'MOD_CONCORRENCIA',
       'QT_VAGAS_CONCORRENCIA', 'PESO_L', 'PESO_CH', 'PESO_CN', 'PESO_M',
       'PESO_R', 'NOTA_MINIMA_L', 'NOTA_MINIMA_CH', 'NOTA_MINIMA_CN',
       'NOTA_MINIMA_M', 'NOTA_MINIMA_R', 'MEDIA_MINIMA', 'INSCRITO', 'SEXO',
          'DT_NASCIMENTO', 'UF_CANDIDATO',
       'MUNICIPIO_CANDIDATO', 'OPCAO', 'NOTA_L', 'NOTA_CH', 'NOTA_CN',
       'NOTA_M', 'NOTA_R', 'NOTA_L_COM_PESO', 'NOTA_CH_COM_PESO',
       'NOTA_CN_COM_PESO', 'NOTA_M_COM_PESO', 'NOTA_R_COM_PESO',
       'NOTA_CANDIDATO', 'NOTA_CORTE','CLASSIFICACAO', 'APROVADO', 'MATRICULA']

In [ ]:
dados = dados[coluns]

In [ ]:
dados.select('*').show()

+----+--------------------+------+--------------------+---------+----------------+--------------------+-----------+-------+----------------+---------------------+------------------+---------------------+------+-------+-------+------+------+-------------+--------------+--------------+-------------+-------------+------------+--------------------+----+-------------+------------+-------------------+-----+------+-------+-------+------+------+---------------+----------------+----------------+---------------+---------------+--------------+----------+-------------+--------+--------------+
| ANO|            NOME_IES|UF_IES|         NOME_CAMPUS|UF_CAMPUS|MUNICIPIO_CAMPUS|          NOME_CURSO|       GRAU|  TURNO|DS_PERIODICIDADE|TIPO_MOD_CONCORRENCIA|  MOD_CONCORRENCIA|QT_VAGAS_CONCORRENCIA|PESO_L|PESO_CH|PESO_CN|PESO_M|PESO_R|NOTA_MINIMA_L|NOTA_MINIMA_CH|NOTA_MINIMA_CN|NOTA_MINIMA_M|NOTA_MINIMA_R|MEDIA_MINIMA|            INSCRITO|SEXO|DT_NASCIMENTO|UF_CANDIDATO|MUNICIPIO_CANDIDATO|OPCAO|NOTA_L|N

In [ ]:
cursos = [
    'MEDICINA', 'DIREITO',
    'ENGENHARIA ELÉTRICA',
    'ARQUITETURA E URBANISMO',
    'PSICOLOGIA',
    'CIÊNCIA DA COMPUTAÇÃO',
]

In [ ]:
base = dados.select(coluns).filter((dados.NOME_CURSO == 'MEDICINA')
  | (dados.NOME_CURSO == 'DIREITO')
  | (dados.NOME_CURSO == 'ENGENHARIA ELÉTRICA')
  | (dados.NOME_CURSO == 'CIÊNCIA DA COMPUTAÇÃO')
  | (dados.NOME_CURSO == 'ARQUITETURA E URBANISMO')
  | (dados.NOME_CURSO== 'PSICOLOGIA')
)

In [ ]:
# base.select('*').show()
base.count(), len(base.columns)

(225950, 45)

In [ ]:
base.select(col('NOME_CURSO')).distinct().show(truncate=False)

+-----------------------+
|NOME_CURSO             |
+-----------------------+
|CIÊNCIA DA COMPUTAÇÃO  |
|PSICOLOGIA             |
|MEDICINA               |
|ENGENHARIA ELÉTRICA    |
|DIREITO                |
|ARQUITETURA E URBANISMO|
+-----------------------+



In [ ]:
base.columns

['ANO',
 'NOME_IES',
 'UF_IES',
 'NOME_CAMPUS',
 'UF_CAMPUS',
 'MUNICIPIO_CAMPUS',
 'NOME_CURSO',
 'GRAU',
 'TURNO',
 'DS_PERIODICIDADE',
 'TIPO_MOD_CONCORRENCIA',
 'MOD_CONCORRENCIA',
 'QT_VAGAS_CONCORRENCIA',
 'PESO_L',
 'PESO_CH',
 'PESO_CN',
 'PESO_M',
 'PESO_R',
 'NOTA_MINIMA_L',
 'NOTA_MINIMA_CH',
 'NOTA_MINIMA_CN',
 'NOTA_MINIMA_M',
 'NOTA_MINIMA_R',
 'MEDIA_MINIMA',
 'INSCRITO',
 'SEXO',
 'DT_NASCIMENTO',
 'UF_CANDIDATO',
 'MUNICIPIO_CANDIDATO',
 'OPCAO',
 'NOTA_L',
 'NOTA_CH',
 'NOTA_CN',
 'NOTA_M',
 'NOTA_R',
 'NOTA_L_COM_PESO',
 'NOTA_CH_COM_PESO',
 'NOTA_CN_COM_PESO',
 'NOTA_M_COM_PESO',
 'NOTA_R_COM_PESO',
 'NOTA_CANDIDATO',
 'NOTA_CORTE',
 'CLASSIFICACAO',
 'APROVADO',
 'MATRICULA']

In [ ]:
base.select('*').show(n=5)

+----+--------------------+------+--------------------+---------+----------------+--------------------+-----------+--------+----------------+---------------------+------------------+---------------------+------+-------+-------+------+------+-------------+--------------+--------------+-------------+-------------+------------+--------------------+----+-------------+------------+-------------------+-----+------+-------+-------+------+------+---------------+----------------+----------------+---------------+---------------+--------------+----------+-------------+--------+---------+
| ANO|            NOME_IES|UF_IES|         NOME_CAMPUS|UF_CAMPUS|MUNICIPIO_CAMPUS|          NOME_CURSO|       GRAU|   TURNO|DS_PERIODICIDADE|TIPO_MOD_CONCORRENCIA|  MOD_CONCORRENCIA|QT_VAGAS_CONCORRENCIA|PESO_L|PESO_CH|PESO_CN|PESO_M|PESO_R|NOTA_MINIMA_L|NOTA_MINIMA_CH|NOTA_MINIMA_CN|NOTA_MINIMA_M|NOTA_MINIMA_R|MEDIA_MINIMA|            INSCRITO|SEXO|DT_NASCIMENTO|UF_CANDIDATO|MUNICIPIO_CANDIDATO|OPCAO|NOTA_L|NOTA

In [ ]:
base.select([count(when(col(c).isNull(), c)).alias(c) for c in base.columns]).show()

+---+--------+------+-----------+---------+----------------+----------+----+-----+----------------+---------------------+----------------+---------------------+------+-------+-------+------+------+-------------+--------------+--------------+-------------+-------------+------------+--------+----+-------------+------------+-------------------+-----+------+-------+-------+------+------+---------------+----------------+----------------+---------------+---------------+--------------+----------+-------------+--------+---------+
|ANO|NOME_IES|UF_IES|NOME_CAMPUS|UF_CAMPUS|MUNICIPIO_CAMPUS|NOME_CURSO|GRAU|TURNO|DS_PERIODICIDADE|TIPO_MOD_CONCORRENCIA|MOD_CONCORRENCIA|QT_VAGAS_CONCORRENCIA|PESO_L|PESO_CH|PESO_CN|PESO_M|PESO_R|NOTA_MINIMA_L|NOTA_MINIMA_CH|NOTA_MINIMA_CN|NOTA_MINIMA_M|NOTA_MINIMA_R|MEDIA_MINIMA|INSCRITO|SEXO|DT_NASCIMENTO|UF_CANDIDATO|MUNICIPIO_CANDIDATO|OPCAO|NOTA_L|NOTA_CH|NOTA_CN|NOTA_M|NOTA_R|NOTA_L_COM_PESO|NOTA_CH_COM_PESO|NOTA_CN_COM_PESO|NOTA_M_COM_PESO|NOTA_R_COM_PESO|NOTA

In [ ]:
base.count()

225950

In [ ]:
base.filter((col('NOTA_CORTE').isNull()) ).select('NOTA_CORTE','CLASSIFICACAO').show()

+----------+-------------+
|NOTA_CORTE|CLASSIFICACAO|
+----------+-------------+
|      NULL|         NULL|
|      NULL|            1|
|      NULL|         NULL|
|      NULL|            3|
|      NULL|            1|
|      NULL|            2|
|      NULL|            2|
|      NULL|         NULL|
|      NULL|         NULL|
|      NULL|         NULL|
|      NULL|            1|
|      NULL|            4|
|      NULL|            3|
|      NULL|            1|
|      NULL|            2|
|      NULL|            3|
|      NULL|            1|
|      NULL|            1|
|      NULL|            1|
|      NULL|         NULL|
+----------+-------------+
only showing top 20 rows



In [ ]:
base = base.dropna(subset=['CLASSIFICACAO', 'NOTA_CORTE'])

In [ ]:
base.count()

220780

In [ ]:
base.filter((col('NOTA_CORTE').isNull()) ).select('NOTA_CORTE','CLASSIFICACAO').show()

+----------+-------------+
|NOTA_CORTE|CLASSIFICACAO|
+----------+-------------+
+----------+-------------+



## Salvar os dados no Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
d = base.toPandas()

In [ ]:
# Salvar no Google Drive
# d.to_csv('/content/drive/MyDrive/RData_Projeto_ENEM_2023_2/Dados/sisu_transformado.csv', index=False)
